# Pobieranie danych
Zaczynam od zgrania pliku z [Lab of European Economics LUISS](https://euklems-intanprod-llee.luiss.it/download/) - to ok. 200 MB CSV - growth accounts


In [ ]:
!wget -O growth-accounts.csv "https://www.dropbox.com/scl/fi/vw1drt9u8i5vcqtrhqbxx/growth-accounts.csv?rlkey=1tfoq18uo9vtkadhx24p3tx59&dl=1"

In [ ]:
import pandas as pd

df = pd.read_csv("growth-accounts.csv")

In [ ]:
df_total = df[df.nace_r2_code == "TOT"]
df_total.head()

# Wykres 1: Indeksy produktywności
Na początku zestawiam indeks produktywności w przypadku wartości dodanej:

In [ ]:
df_indices = df_total[["geo_code", "year", "var", "value"]].copy()
df_indices = df_indices[df_indices["var"] == "VATFP_I"]
df_indices

Tabela przestawna ustawia nazwę krajów:

In [ ]:
df_indices_pivot = df_indices.pivot(index="year", columns="geo_code", values="value")
df_indices_pivot

Na wykres wybieram wartość z 2021

In [ ]:
df_chart = df_indices_pivot.loc[2021]
df_chart = df_chart.sort_values(ascending=False)
df_chart = df_chart - 100
df_chart.to_csv("TFP_indices.csv")

Rozpisane jako funkcja aby zbierać po sektorach:

In [ ]:
def compile_TFP_indices(df):
  df_indices = df[["geo_code", "year", "var", "value"]].copy()
  df_indices = df_indices[df_indices["var"] == "VATFP_I"]

  df_indices_pivot = df_indices.pivot(index="year", columns="geo_code", values="value")

  df_chart = df_indices_pivot.loc[2021]
  df_chart = df_chart.sort_values(ascending=False)
  df_chart = df_chart - 100
  return df_chart

In [ ]:
df_indu = compile_TFP_indices(df[df.nace_r2_code == "C"])
df_indu

## Wykres 2 - zestawienie produktywności
Zaczynamy od przygototowania funkcji, która stworzy nam gotową ramkę z dekompozycją kontrybucji:

In [8]:
def get_value_added_decomposition(df, code:str):
  variable_list = ["VAConH", "VAConLC", "VAConTangNICT",
                   "VAConTangICT", "VAConIntang", "VAConTFP", "VA_G"]

  # Select variables:
  df_indices = df[["geo_code", "year", "var", "value"]].copy()
  df_indices = df_indices[df_indices['var'].isin(variable_list)]

  # Select country:
  df_indices = df_indices[df_indices.geo_code == code]
  df_indices = df_indices.drop(columns="geo_code")

  df_indices = df_indices.pivot(index="year", columns="var", values="value")
  return df_indices

Przygotowanie ramki i zapis do XLS dla odpowiednich krajów.

In [12]:
codes = ["PL", "DE", "CZ", "SK", "HU"]

for code in codes:
  df_value_added = get_value_added_decomposition(df_total, code)
  df_value_added.to_excel("value_added_decomposition_" + code + ".xlsx")